In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

# Choix des $q_k$ aléatoires

In [290]:
def next_matrix(Q, w0, x0, w, x, max_iter):
    N = len(x)
    Q_tilde = np.zeros((N+1, N+1))
    Q_tilde[1:, 1:] = Q
    Q_tilde[N, 0] = w0
    
    x_tilde = np.zeros(N+1)
    x_tilde[0] = x0
    for i in range(N):
        x_tilde[i+1] = x[i]

    it = 0
    while(it < max_iter):
        is_compatible = True
        q = 100*np.random.randn(N)

        Q_tilde[0, 0] = - np.sum(q)
        for i in range(N):
            Q_tilde[0, i+1] = q[i]

        # vérification de la compatibilité
        Q_tilde_inv = np.linalg.inv(Q_tilde)
        
        # [Q_tilde Diag(x0,x1,...,xN) Q_tilde-1]ij <= 0 pour i != j
        D_tilde = np.diag(x_tilde)
        A = Q_tilde @ D_tilde @ Q_tilde_inv
        for i in range(N+1):
            for j in range(N+1):
                if(i != j and A[i, j] > 0):
                    is_compatible = False
        it += 1
        
        if(is_compatible):
            return A, Q_tilde
    return None, None

In [291]:
x = np.array([9, 10, 11])
w = np.array([5, 2, 10])

b = 1
y1 = x[1]-x[0]
y2 = x[2]- x[1]
a = (y1+y2) / y2
Q = np.array([[1, -a, -1+a],
              [1, b, -1-b],
              [w[0], w[1], w[2]]])

x0 = 1
w0 = 10

In [294]:
A, Q_tilde = next_matrix(Q, w0, x0, w, x, 1000000)
print(A)
print(Q_tilde)

None
None


# Optimisation pour trouver des $q_k$ convenables

In [35]:
def compute_A(Q, w0, x0, x, q):
    N = len(x)

    # Construction de Q_tilde (matrice (N+1)x(N+1))
    Q_tilde = np.zeros((N + 1, N + 1))
    Q_tilde[1:, 1:] = Q
    Q_tilde[0, 0] = -np.sum(q)
    Q_tilde[0, 1:] = q
    Q_tilde[N, 0] = w0  # note : w n'est pas utilisé ici

    # Construction de x_tilde (vecteur N+1)
    x_tilde = np.zeros(N + 1)
    x_tilde[0] = x0
    x_tilde[1:] = x

    try:
        Q_tilde_inv = np.linalg.inv(Q_tilde)
    except np.linalg.LinAlgError:
        raise ValueError("Q_tilde is not invertible.")

    # Calcul de A = Q_tilde @ diag(x_tilde) @ Q_tilde_inv
    D_tilde = np.diag(x_tilde)
    A = Q_tilde @ D_tilde @ Q_tilde_inv

    return A

def phi(q, Q, w0, x0, w, x):
    A = compute_A(Q, w0, x0, x, q)
    N = A.shape[0]
    total = 0.0
    for i in range(N):
        for j in range(N):
            if i != j:
                if A[i, j] > 0:
                    total += A[i, j]
                else:
                    total += A[i, j]
    return total

def optimize_q_unconstrained(Q, w0, x0, w, x):
    N = len(x)
    
    def objective(q):
        return phi(q, Q, w0, x0, w, x)
    
    q0 = np.random.randn(N)  # Initialisation sans normalisation
    
    res = minimize(
        objective,
        x0=q0,
        method='BFGS',  # Méthode sans contraintes
        options={'disp': True}
    )
    
    if res.success:
        print("Optimisation réussie")
    else:
        print("Échec de l'optimisation :", res.message)
    
    return res.x, res.fun

q_opt, phi_val = optimize_q_unconstrained(Q, w0, x0, w, x)
print("q optimal :", q_opt)
print("phi(q_opt) :", phi_val)


         Current function value: -1544.720227
         Iterations: 2
         Function evaluations: 464
         Gradient evaluations: 113
Échec de l'optimisation : Desired error not necessarily achieved due to precision loss.
q optimal : [ 542.28644952 -298.51948752  453.01291108]
phi(q_opt) : -1544.720226733944


c:\Users\Maxence CAUCHETEUX\anaconda3\Lib\site-packages\scipy\optimize\_minimize.py:733: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


# À la main

In [44]:
def check_neg(Q_tilde, w0, x0, w, x):
    N = len(Q_tilde) - 1
    x_tilde = np.zeros(N + 1)
    x_tilde[0] = x0
    x_tilde[1:] = x
    D = np.diag(x_tilde)
    Q_tilde_inv = np.linalg.inv(Q_tilde)
    M = Q_tilde @ D @ Q_tilde_inv
    print("Q_tilde D Q_tilde^-1 :")
    print(M)
    print('\n')
    for i in range(N+1):
        for j in range(N+1):
            if(i!=j and M[i,j] > 0):
                return False
    return True


In [46]:
q1 = -1.2
q2 = -0.6  
q3 =  1.4

N = 3

Q_tilde = np.zeros((N+1, N+1))
Q_tilde[1:, 1:] = Q
Q_tilde[N, 0] = w0
Q_tilde[0,1] = q1
Q_tilde[0,2] = q2
Q_tilde[0, 3] = q3
Q_tilde[0,0] = -q1-q2-q3

print("Q_tilde :")
print(Q_tilde)
print("\n")

print(check_neg(Q_tilde, w0, x0, w, x))

Q_tilde :
[[ 0.4 -1.2 -0.6  1.4]
 [ 0.   1.  -1.   0. ]
 [ 0.   1.   2.  -3. ]
 [ 2.   1.   2.   3. ]]


Q_tilde D Q_tilde^-1 :
[[-19.175      -11.305      -16.76         3.855     ]
 [  2.5          3.83333333   0.66666667  -0.5       ]
 [ 40.          21.33333333  35.66666667  -8.        ]
 [-53.375      -34.69166667 -39.13333333  10.775     ]]


False
